# Hate speech classification task

## Veysel Kaan Bati

In [ ]:
import keras
import keras_nlp
import random
import re
import string
import numpy as np
import pandas as pd
import tensorflow as tf
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.optimizers import Adam
from keras.metrics import SparseCategoricalAccuracy
from keras.losses import SparseCategoricalCrossentropy
from keras.models import Sequential
from keras.layers import Dense



In [49]:
url_data = ('/compLing/students/courses/deepLearning/assignment02/labeled_data.csv')
data = pd.read_csv(url_data)
data.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [50]:
filtered_data = data[["class", "tweet"]]

In [51]:
def preprocess_tweet(tweet):
    tweet = tweet.lower()
    # Remove URLs
    tweet = re.sub(r'http\S+', '', tweet)
    tweet = re.sub(r'@\S+', '', tweet)
    # Remove special characters and punctuation
    tweet = re.sub(r'[^a-zA-Z\s]', '', tweet)
    tweet = re.sub(r'amp', '', tweet)
    tweet = re.sub(r'rt', '', tweet)
    tweet = re.sub(' +', ' ', tweet)
    tweet = tweet.replace('\n', '')
    return tweet

In [52]:
filtered_data['tweet'] = filtered_data['tweet'].apply(lambda x: preprocess_tweet(x))

/tmp/ipykernel_49351/946608730.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['tweet'] = filtered_data['tweet'].apply(lambda x: preprocess_tweet(x))


In [54]:
filtered_data.head()

,class,tweet
0,2,as a woman you shouldnt complain about cleani...
1,1,boy dats coldtyga dwn bad for cuffin dat hoe ...
2,1,dawg you ever fuck a bitch and she sta to cry...
3,1,she look like a tranny
4,1,the shit you hear about me might be true or i...


In [9]:
train_df, temp_df = train_test_split(filtered_data, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Print the shapes of the resulting DataFrames
print("Train set shape:", train_df.shape)
print("Validation set shape:", val_df.shape)
print("Test set shape:", test_df.shape)

Train set shape: (19826, 2)
Validation set shape: (2478, 2)
Test set shape: (2479, 2)


In [10]:
train_df.head()

,class,tweet
15272,0,well how else will white ppl get us to forget...
9351,2,funny thing isits not just the people doing it...
20323,1,nigga messed with the wrong bitch
3638,1,bitch ass nigggaaa
20579,1,so that real bitch


In [11]:
max_words = 10000
max_len = 50

In [ ]:
text_vectorization = keras.layers.TextVectorization(
    max_tokens=max_words,
    output_mode="int",
    output_sequence_length=max_len)

In [15]:
text_vectorization.adapt(train_df["tweet"])

In [16]:
t_train_vec = text_vectorization(train_df["tweet"])
t_val_vec = text_vectorization(val_df["tweet"])
t_test_vec = text_vectorization(test_df["tweet"])


In [17]:
vocabulary = text_vectorization.get_vocabulary()
test_sentence = filtered_data["tweet"][0]
encoded_sentence = text_vectorization(test_sentence)
print(encoded_sentence)
inverse_vocab = dict(enumerate(vocabulary))
decoded_sentence = " ".join(inverse_vocab[int(i)] for i in encoded_sentence)
print(decoded_sentence)

tf.Tensor(
[  72    2  352    6 1158 1197   59 3580   30   29  296   72    2   89
    6  192  167  146    5   53   55    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0], shape=(50,), dtype=int64)
as a woman you shouldnt complain about cleaning up your house as a man you should always take the trash out                             


In [18]:
text_vectorization.get_vocabulary()[:10]

['', '[UNK]', 'a', 'bitch', 'i', 'the', 'you', 'to', 'and', 'my']

In [19]:
glove_path = '/compLing/students/courses/deepLearning/assignment02/glove.twitter.27B.100d.txt'
import numpy as np
embeddings_index = {}
with open(glove_path) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs
print(f"Found {len(embeddings_index)} word vectors.")


Found 1193514 word vectors.


In [20]:
embedding_dim = 100
vocabulary = text_vectorization.get_vocabulary()
word_index = dict(zip(vocabulary, range(len(vocabulary))))
num_words = min(max_words, len(word_index) + 1)
embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [21]:
input_layer = keras.layers.Input(shape=(max_len,))
tok = keras.layers.Embedding(input_dim=text_vectorization.vocabulary_size(),
                           output_dim=embedding_dim,
                           weights=[embedding_matrix],
                           trainable=False)(input_layer)

position_embeddings = keras_nlp.layers.PositionEmbedding(
    sequence_length=max_len)(tok)
x = tok + position_embeddings

x = keras_nlp.layers.TransformerEncoder(intermediate_dim=64, num_heads=4)(x)
x = keras.layers.GlobalAveragePooling1D()(x)
x = keras.layers.Dropout(0.5)(x)

output = keras.layers.Dense(3, activation="softmax")(x)

In [22]:
model = keras.Model(input_layer, output)

In [23]:
optimizer = Adam()  # You can also use 'rmsprop' if preferred
model.compile(optimizer=optimizer, loss=SparseCategoricalCrossentropy(), metrics=[SparseCategoricalAccuracy()])

In [24]:
callbacks = [
    keras.callbacks.ModelCheckpoint("hate_detect.tf",
                                    save_best_only=True)
] 

In [25]:
model.fit(t_train_vec, train_df["class"], epochs=10, batch_size=64, validation_data=(t_val_vec, val_df["class"]), callbacks=callbacks)

model =  keras.models.load_model("hate_detect.tf")

Epoch 1/10
309/310 [============================>.] - ETA: 0s - loss: 0.3621 - sparse_categorical_accuracy: 0.8770

INFO:tensorflow:Assets written to: hate_detect.tf/assets


INFO:tensorflow:Assets written to: hate_detect.tf/assets


310/310 [==============================] - 13s 38ms/step - loss: 0.3620 - sparse_categorical_accuracy: 0.8768 - val_loss: 0.2956 - val_sparse_categorical_accuracy: 0.8890
Epoch 2/10
309/310 [============================>.] - ETA: 0s - loss: 0.2773 - sparse_categorical_accuracy: 0.9036

INFO:tensorflow:Assets written to: hate_detect.tf/assets


...

INFO:tensorflow:Assets written to: hate_detect.tf/assets


310/310 [==============================] - 11s 36ms/step - loss: 0.2586 - sparse_categorical_accuracy: 0.9076 - val_loss: 0.2814 - val_sparse_categorical_accuracy: 0.8935
Epoch 5/10
310/310 [==============================] - 10s 31ms/step - loss: 0.2535 - sparse_categorical_accuracy: 0.9089 - val_loss: 0.2839 - val_sparse_categorical_accuracy: 0.8947
Epoch 6/10
309/310 [============================>.] - ETA: 0s - loss: 0.2477 - sparse_categorical_accuracy: 0.9106

INFO:tensorflow:Assets written to: hate_detect.tf/assets


INFO:tensorflow:Assets written to: hate_detect.tf/assets


310/310 [==============================] - 11s 36ms/step - loss: 0.2474 - sparse_categorical_accuracy: 0.9108 - val_loss: 0.2752 - val_sparse_categorical_accuracy: 0.9048
Epoch 7/10
310/310 [==============================] - 10s 32ms/step - loss: 0.2443 - sparse_categorical_accuracy: 0.9104 - val_loss: 0.2900 - val_sparse_categorical_accuracy: 0.8999
Epoch 8/10
310/310 [==============================] - 10s 31ms/step - loss: 0.2384 - sparse_categorical_accuracy: 0.9130 - val_loss: 0.2821 - val_sparse_categorical_accuracy: 0.8995
Epoch 9/10
310/310 [==============================] - 10s 32ms/step - loss: 0.2353 - sparse_categorical_accuracy: 0.9146 - val_loss: 0.2784 - val_sparse_categorical_accuracy: 0.9052
Epoch 10/10
310/310 [==============================] - 10s 32ms/step - loss: 0.2314 - sparse_categorical_accuracy: 0.9172 - val_loss: 0.2795 - val_sparse_categorical_accuracy: 0.9003


In [26]:
y_pred = model.predict(t_test_vec) # x_test here should be your test data
y_pred_bool = np.argmax(y_pred, axis=1)
print(classification_report(test_df["class"], y_pred_bool))

78/78 [==============================] - 1s 10ms/step
              precision    recall  f1-score   support

           0       0.46      0.10      0.16       133
           1       0.93      0.95      0.94      1924
           2       0.80      0.90      0.85       422

    accuracy                           0.90      2479
   macro avg       0.73      0.65      0.65      2479
weighted avg       0.88      0.90      0.88      2479



In [27]:
num_samples = len(filtered_data)
num_samples

24783

In [28]:
avg_words_per_sample = filtered_data['tweet'].apply(lambda x: len(x.split())).mean()
avg_words_per_sample

12.480369608199169

In [29]:
num_samples / avg_words_per_sample

1985.7584973860414

In [30]:
# TF-IDF vectorizer with bigrams
tf_vectorizer = keras.layers.TextVectorization(
    max_tokens=max_words,
    output_mode="tf_idf")

In [31]:
tf_vectorizer.adapt(train_df["tweet"])

In [32]:
tf_train_vec = tf_vectorizer(train_df["tweet"])
tf_val_vec = tf_vectorizer(val_df["tweet"])
tf_test_vec = tf_vectorizer(test_df["tweet"])

In [33]:
model_tf = Sequential()
model_tf.add(Dense(128, activation='relu', input_shape=(tf_train_vec.shape[1],)))
model_tf.add(Dense(3, activation='softmax'))

In [34]:
model_tf.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
model_tf.fit(tf_train_vec, train_df["class"] , epochs=10, batch_size=64, validation_data=(tf_val_vec, val_df["class"]))

Epoch 1/10
310/310 [==============================] - 1s 3ms/step - loss: 0.3907 - sparse_categorical_accuracy: 0.8688 - val_loss: 0.3441 - val_sparse_categorical_accuracy: 0.8846
Epoch 2/10
310/310 [==============================] - 1s 2ms/step - loss: 0.1286 - sparse_categorical_accuracy: 0.9552 - val_loss: 0.4102 - val_sparse_categorical_accuracy: 0.8785
Epoch 3/10
310/310 [==============================] - 1s 2ms/step - loss: 0.0597 - sparse_categorical_accuracy: 0.9808 - val_loss: 0.4779 - val_sparse_categorical_accuracy: 0.8753
Epoch 4/10
310/310 [==============================] - 1s 3ms/step - loss: 0.0349 - sparse_categorical_accuracy: 0.9894 - val_loss: 0.5339 - val_sparse_categorical_accuracy: 0.8717
Epoch 5/10
310/310 [==============================] - 1s 3ms/step - loss: 0.0235 - sparse_categorical_accuracy: 0.9925 - val_loss: 0.6027 - val_sparse_categorical_accuracy: 0.8733
Epoch 6/10
310/310 [==============================] - 1s 2ms/step - loss: 0.0178 - sparse_categorica

In [38]:
y_pred = model_tf.predict(tf_test_vec) 
y_pred_bool = np.argmax(y_pred, axis=1)
print(classification_report(test_df["class"], y_pred_bool))

78/78 [==============================] - 0s 903us/step
              precision    recall  f1-score   support

           0       0.31      0.19      0.23       133
           1       0.90      0.95      0.92      1924
           2       0.82      0.73      0.77       422

    accuracy                           0.87      2479
   macro avg       0.68      0.62      0.64      2479
weighted avg       0.86      0.87      0.86      2479

